# 🚀 CenQuery: Llama 3 8B LoRA Fine-Tuning
**Project:** Indian Census Text-to-SQL  
**Hardware:** Runs on T4 GPU (Free Colab Tier)

This notebook fine-tunes the `defog/llama-3-sqlcoder-8b` model on your custom Census dataset using QLoRA (4-bit quantization).

## 1. Install Dependencies
We need specific versions of `peft`, `bitsandbytes`, and `transformers` to run 4-bit training.

In [1]:
# Install compatible versions of all libraries
# !pip install -U torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U bitsandbytes
!pip install -q -U trl
!pip install -q -U accelerate
!pip install -q -U datasets scipy

print("✅ Installation complete. Please RESTART the session now.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.6 MB/s eta 0:00:00
✅ Installation complete. Please RESTART the session now.


## 2. Upload Training Data
Upload the **`consolidated_train.jsonl`** file that your team created.

In [ ]:
from google.colab import files
import os

# Check if file already exists to avoid re-uploading
if not os.path.exists('consolidated_train.jsonl'):
    print("📂 Please upload 'consolidated_train.jsonl'...")
    uploaded = files.upload()
    # Rename if necessary to ensure it matches the expected filename
    for filename in uploaded.keys():
        if filename != 'consolidated_train.jsonl':
            os.rename(filename, 'consolidated_train.jsonl')
            print(f"   Renamed {filename} to consolidated_train.jsonl")
else:
    print("✅ 'consolidated_train.jsonl' found.")

📂 Please upload 'consolidated_train.jsonl'...


Saving train_sourish_final.jsonl to train_sourish_final.jsonl
   Renamed train_sourish_final.jsonl to consolidated_train.jsonl


## 3. Run QLoRA Training
This script loads the 8B model in 4-bit mode and fine-tunes it on your data.

In [ ]:
def train():
    print(f"🚀 Initializing Training for {MODEL_NAME}...")

    # 1. Quantization Config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    # 2. Load Base Model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        use_cache=False
    )
    model.config.pretraining_tp = 1

    # 3. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # 4. Load Dataset
    if not os.path.exists(TRAIN_DATA_PATH):
        print(f"❌ Error: {TRAIN_DATA_PATH} not found.")
        return None

    dataset = load_dataset("json", data_files=TRAIN_DATA_PATH, split="train")
    print(f"✅ Loaded {len(dataset)} training examples.")

    # 5. LoRA Config
    peft_config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )

    # 6. SFT Configuration (Replaces TrainingArguments + max_seq_length arg)
    sft_config = SFTConfig(
        output_dir=OUTPUT_DIR,
        dataset_text_field="text",  # Newer trl versions support this in SFTConfig
        # max_seq_length=MAX_SEQ_LENGTH,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUMULATION,
        optim="paged_adamw_32bit",
        save_steps=25,
        logging_steps=5,
        learning_rate=LEARNING_RATE,
        weight_decay=0.001,
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="constant",
        report_to="none",
        packing=False
    )

    # 7. Trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        # tokenizer=tokenizer,
        max_seq_length=MAX_SEQ_LENGTH
        args=sft_config, # Pass SFTConfig here
    )

    # 8. Train
    print("🔥 Starting Training...")
    tt = trainer.train()

    # 9. Save Locally
    print(f"💾 Saving adapter locally to {NEW_MODEL_NAME}...")
    trainer.save_model(NEW_MODEL_NAME)
    tokenizer.save_pretrained(NEW_MODEL_NAME)
    return trainer

# Run the function
trainer = train()

🚀 Initializing Training for defog/llama-3-sqlcoder-8b...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Loaded 150 training examples.


...

Truncating train dataset:   0%|          | 0/150 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009}.


🔥 Starting Training...


Step,Training Loss
5,0.514900
10,0.040500
15,0.020600


💾 Saving adapter locally to llama-3-8b-census-sql-adapter...


### new kind not tested yet

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments, # Reverted to standard TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer
import os

# --- Configuration ---
MODEL_NAME = "defog/llama-3-sqlcoder-8b"
NEW_MODEL_NAME = "llama-3-8b-census-sql-adapter"
TRAIN_DATA_PATH = "consolidated_train.jsonl"
OUTPUT_DIR = "./results"

# LoRA Params
LORA_R = 32
LORA_ALPHA = 64
LORA_DROPOUT = 0.05

# Training Params
NUM_EPOCHS = 1
BATCH_SIZE = 2
GRAD_ACCUMULATION = 4
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 2048

def train():
    print(f"🚀 Initializing Training for {MODEL_NAME}...")

    # 1. Quantization Config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

    # 2. Load Base Model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        use_cache=False
    )
    model.config.pretraining_tp = 1

    # 3. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # 4. Load Dataset
    if not os.path.exists(TRAIN_DATA_PATH):
        print(f"❌ Error: {TRAIN_DATA_PATH} not found.")
        return None

    dataset = load_dataset("json", data_files=TRAIN_DATA_PATH, split="train")
    print(f"✅ Loaded {len(dataset)} training examples.")

    # 5. LoRA Config
    peft_config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )

    # 6. Training Arguments (Standard, robust configuration)
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUMULATION,
        optim="paged_adamw_32bit",
        save_steps=25,
        logging_steps=5,
        learning_rate=LEARNING_RATE,
        weight_decay=0.001,
        fp16=False,
        bf16=False,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="constant",
        report_to="none",
        # NOTE: SFTTrainer specific args like max_seq_length are NOT passed here
    )

    # 7. Trainer
    # Formatting function is the key fix for modern trl versions
    def formatting_prompts_func(example):
        output_texts = []
        for text in example['text']:
            output_texts.append(text)
        return output_texts

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=peft_config,
        tokenizer=tokenizer,
        args=training_args,        # Pass standard TrainingArgs
        max_seq_length=MAX_SEQ_LENGTH, # Pass max_seq_length explicitly to Trainer
        formatting_func=formatting_prompts_func, # Explicit formatting
        packing=False,
    )

    # 8. Train
    print("🔥 Starting Training...")
    trainer.train()

    # 9. Save Locally (FIXED: Use trainer.save_model instead of trainer.model.save_pretrained)
    print(f"💾 Saving adapter locally to {NEW_MODEL_NAME}...")

    # This method handles saving the adapter and tokenizer correctly
    trainer.save_model(NEW_MODEL_NAME)

    # Just to be safe, we can also try saving the PEFT model explicitly if needed,
    # but trainer.save_model() is the preferred high-level API.
    # If you specifically need just the adapter:
    # trainer.model.save_pretrained(NEW_MODEL_NAME)

    return trainer

if __name__ == "__main__":
    train()

## 4. Save Adapter to Google Drive
Mount your Google Drive to save the trained model permanently. This way, if Colab disconnects, you don't lose your work.

In [ ]:
from google.colab import drive
import shutil

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Define Destination
DESTINATION_FOLDER = "/content/drive/MyDrive/Major_Project/CenQuery_Adapter"

# 3. Copy files
print(f"💾 Copying model files to {DESTINATION_FOLDER}...")
if os.path.exists(DESTINATION_FOLDER):
    shutil.rmtree(DESTINATION_FOLDER)
shutil.copytree("llama-3-8b-census-sql-adapter", DESTINATION_FOLDER)

print("✅ Success! Adapter saved to Google Drive.")

Mounted at /content/drive
💾 Copying model files to /content/drive/MyDrive/Major_Project/CenQuery_Adapter...
✅ Success! Adapter saved to Google Drive.


## 5. Verify it

In [3]:
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# 2. Define Destination
FOLDER = "/content/drive/MyDrive/Major_Project"
adapter = "/CenQuery_Adapter"
model = "/models/llama-3-sqlcoder-8b"

import gc, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel


BASE_MODEL_NAME = FOLDER + model
ADAPTER_PATH = FOLDER + adapter


In [20]:
schema_context = """
    CREATE TABLE regions (state BIGINT PRIMARY KEY, area_name TEXT);
    CREATE TABLE tru (id BIGINT PRIMARY KEY, name TEXT);
    CREATE TABLE religions (id BIGINT PRIMARY KEY, religion_name TEXT);
    CREATE TABLE languages (id BIGINT PRIMARY KEY, name TEXT);
    CREATE TABLE age_groups (id BIGINT PRIMARY KEY, name TEXT);
    CREATE TABLE population_stats (state BIGINT, tru_id BIGINT, age TEXT, persons BIGINT, males BIGINT, females BIGINT);
    CREATE TABLE healthcare_stats (state BIGINT, tru_id BIGINT, number_of_hh_surveyed BIGINT, number_of_women_15_49_interviewed BIGINT, number_of_men_15_54_interviewed BIGINT, female_pop_age_6_years_and_above_who_ever_school_ DOUBLE PRECISION, pop_below_age_15_years_ DOUBLE PRECISION, sex_ratio_of_the_total_pop_females_per_1000_males DOUBLE PRECISION, sex_ratio_at_birth_for_children_born_in_the_last_five_years_ TEXT, child_u5_whose_birth_was_registered_with_the_civil_authority TEXT, deaths_in_the_last_3_years_registered_with_the_civil_authori TEXT, pop_living_in_hh_with_electricity_ DOUBLE PRECISION, pop_living_in_hh_with_an_improved_drinkingwater_source1_ DOUBLE PRECISION, pop_living_in_hh_that_use_an_improved_sanitation_facility2_ DOUBLE PRECISION, hh_using_clean_fuel_for_cooking3_ TEXT, hh_using_iodized_salt_ TEXT, hh_with_any_usual_member_covered_under_a_health_insurancefin TEXT, children_age_5_years_who_attended_preprimary_school_during_t TEXT, women_age_1549_who_are_lit4_ DOUBLE PRECISION, men_age_1549_who_are_lit4_ TEXT, women_age_1549_with_10_or_more_years_of_schooling_ DOUBLE PRECISION, men_age_1549_with_10_or_more_years_of_schooling_ TEXT, women_age_1549_who_have_ever_used_the_internet_ TEXT, men_age_1549_who_have_ever_used_the_internet_ TEXT, women_age_2024_years_married_before_age_18_years_ TEXT, men_age_2529_years_married_before_age_21_years_ TEXT, total_fertility_rate_number_of_children_per_woman TEXT, women_age_1519_years_who_were_already_mothers_or_pregnant_at TEXT, adolescent_fertility_rate_for_women_age_1519_years5 DOUBLE PRECISION, neonatal_mortality_rate_per_1000_live_births TEXT, infant_mortality_rate_per_1000_live_births TEXT, underfive_mortality_rate_per_1000_live_births TEXT, current_use_of_family_planning_methods_currently_married_wom TEXT, current_use_of_family_planning_methods_currently_married_wom.1 TEXT, current_use_of_family_planning_methods_currently_married_wom.2 TEXT, current_use_of_family_planning_methods_currently_married_wom.3 TEXT, current_use_of_family_planning_methods_currently_married_wom.4 TEXT, current_use_of_family_planning_methods_currently_married_wom.5 TEXT, current_use_of_family_planning_methods_currently_married_wom.6 TEXT, current_use_of_family_planning_methods_currently_married_wom.7 TEXT, current_use_of_family_planning_methods_currently_married_wom.8 TEXT, current_use_of_family_planning_methods_currently_married_wom.9 TEXT, current_use_of_family_planning_methods_currently_married_wom.10 TEXT, current_use_of_family_planning_methods_currently_married_wom.11 TEXT, current_use_of_family_planning_methods_currently_married_wom.12 TEXT, current_use_of_family_planning_methods_currently_married_wom.13 TEXT, current_use_of_family_planning_methods_currently_married_wom.14 TEXT, current_use_of_family_planning_methods_currently_married_wom.15 TEXT, current_use_of_family_planning_methods_currently_married_wom.16 TEXT, current_use_of_family_planning_methods_currently_married_wom.17 TEXT, current_use_of_family_planning_methods_currently_married_wom.18 TEXT, current_use_of_family_planning_methods_currently_married_wom.19 TEXT, current_use_of_family_planning_methods_currently_married_wom.20 TEXT, current_use_of_family_planning_methods_currently_married_wom.21 TEXT, current_use_of_family_planning_methods_currently_married_wom.22 TEXT, current_use_of_family_planning_methods_currently_married_wom.23 TEXT, current_use_of_family_planning_methods_currently_married_wom.24 TEXT, current_use_of_family_planning_methods_currently_married_wom.25 TEXT, current_use_of_family_planning_methods_currently_married_wom.26 TEXT, current_use_of_family_planning_methods_currently_married_wom.27 TEXT, current_use_of_family_planning_methods_currently_married_wom.28 TEXT, current_use_of_family_planning_methods_currently_married_wom.29 TEXT, current_use_of_family_planning_methods_currently_married_wom.30 TEXT, current_use_of_family_planning_methods_currently_married_wom.31 TEXT, current_use_of_family_planning_methods_currently_married_wom.32 TEXT, current_use_of_family_planning_methods_currently_married_wom.33 TEXT, current_use_of_family_planning_methods_currently_married_wom.34 TEXT, current_use_of_family_planning_methods_currently_married_wom.35 TEXT, current_use_of_family_planning_methods_currently_married_wom.36 TEXT, current_use_of_family_planning_methods_currently_married_wom.37 TEXT, current_use_of_family_planning_methods_currently_married_wom.38 TEXT, current_use_of_family_planning_methods_currently_married_wom.39 TEXT, current_use_of_family_planning_methods_currently_married_wom.40 TEXT, current_use_of_family_planning_methods_currently_married_wom.41 TEXT, current_use_of_family_planning_methods_currently_married_wom.42 TEXT, current_use_of_family_planning_methods_currently_married_wom.43 TEXT, current_use_of_family_planning_methods_currently_married_wom.44 TEXT, current_use_of_family_planning_methods_currently_married_wom.45 TEXT, current_use_of_family_planning_methods_currently_married_wom.46 TEXT, current_use_of_family_planning_methods_currently_married_wom.47 TEXT, current_use_of_family_planning_methods_currently_married_wom.48 TEXT, current_use_of_family_planning_methods_currently_married_wom.49 TEXT, current_use_of_family_planning_methods_currently_married_wom.50 TEXT, current_use_of_family_planning_methods_currently_married_wom.51 TEXT, current_use_of_family_planning_methods_currently_married_wom.52 TEXT, current_use_of_family_planning_methods_currently_married_wom.53 TEXT, current_use_of_family_planning_methods_currently_married_wom.54 TEXT, current_use_of_family_planning_methods_currently_married_wom.55 TEXT, current_use_of_family_planning_methods_currently_married_wom.56 TEXT, current_use_of_family_planning_methods_currently_married_wom.57 TEXT, current_use_of_family_planning_methods_currently_married_wom.58 TEXT, current_use_of_family_planning_methods_currently_married_wom.59 TEXT, current_use_of_family_planning_methods_currently_married_wom.60 TEXT, current_use_of_family_planning_methods_currently_married_wom.61 TEXT, current_use_of_family_planning_methods_currently_married_wom.62 TEXT, current_use_of_family_planning_methods_currently_married_wom.63 TEXT, total_unmet_need_for_family_planning_currently_married_women TEXT, unmet_need_for_spacing_currently_married_women_age_1549_year TEXT, health_worker_ever_talked_to_female_nonusers_about_family_pl TEXT, current_users_ever_told_about_side_effects_of_current_method TEXT, mothers_who_had_an_antenatal_checkup_in_the_first_trimester_ TEXT, mothers_who_had_at_least_4_antenatal_care_visits_for_last_bi TEXT, mothers_whose_last_birth_was_protected_against_neonatal_teta TEXT, mothers_who_consumed_iron_folic_acid_for_100_days_or_more_wh TEXT, mothers_who_consumed_iron_folic_acid_for_180_days_or_more_wh TEXT, registered_pregnancies_for_which_the_mother_received_a_mothe TEXT, mothers_who_received_postnatal_care_from_a_doctornurselhvanm TEXT, average_outofpocket_expenditure_per_delivery_in_a_public_hea TEXT, children_born_at_home_who_were_taken_to_a_health_facility_fo TEXT, children_who_received_postnatal_care_from_a_doctornurselhvan TEXT, institutional_births_in_the_5_years_before_the_survey_ TEXT, institutional_births_in_public_facility_in_the_5_years_befor TEXT, home_births_that_were_conducted_by_skilled_health_personnel_ TEXT, births_attended_by_skilled_health_personnel_in_the_5_years_b TEXT, births_delivered_by_caesarean_section_in_the_5_years_before_ TEXT, births_in_a_private_health_facility_that_were_delivered_by_c TEXT, births_in_a_public_health_facility_that_were_delivered_by_ca TEXT, children_age_1223_months_fully_vaccinated_based_on_informati TEXT, children_age_1223_months_fully_vaccinated_based_on_informati.1 TEXT, children_age_1223_months_fully_vaccinated_based_on_informati.2 TEXT, children_age_1223_months_fully_vaccinated_based_on_informati.3 TEXT, children_age_1223_months_who_have_received_bcg_ TEXT, children_age_1223_months_who_have_received_3_doses_of_polio_ TEXT, children_age_1223_months_who_have_received_3_doses_of_penta_ TEXT, children_age_1223_months_who_have_received_3_doses_of_penta_.1 TEXT, children_age_1223_months_who_have_received_the_first_dose_of TEXT, children_age_2435_months_who_have_received_a_second_dose_of_ TEXT, children_age_1223_months_who_have_received_3_doses_of_rotavi TEXT, children_age_1223_months_who_have_received_3_doses_of_penta_.2 TEXT, children_age_1223_months_who_have_received_3_doses_of_penta_.3 TEXT, children_age_935_months_who_received_a_vitamin_a_dose_in_the TEXT, children_age_1223_months_who_received_most_of_their_vaccinat TEXT, children_age_1223_months_who_received_most_of_their_vaccinat.1 TEXT, children_age_1223_months_who_received_most_of_their_vaccinat.2 TEXT, children_age_1223_months_who_received_most_of_their_vaccinat.3 TEXT, prevalence_of_diarrhoea_in_the_2_weeks_preceding_the_survey_ TEXT, children_with_diarrhoea_in_the_2_weeks_preceding_the_survey_ TEXT, children_with_diarrhoea_in_the_2_weeks_preceding_the_survey_.1 TEXT, children_with_diarrhoea_in_the_2_weeks_preceding_the_survey_.2 TEXT, children_with_diarrhoea_in_the_2_weeks_preceding_the_survey_.3 TEXT, children_swith_diarrhoea_in_the_2_weeks_preceding_the_survey TEXT, children_prevalence_of_symptoms_of_acute_respiratory_infecti TEXT, children_with_fever_or_symptoms_of_ari_in_the_2_weeks_preced TEXT, children_under_age_3_years_breastfed_within_one_hour_of_birt TEXT, children_under_age_6_months_exclusively_breastfed16_ TEXT, children_age_68_months_receiving_solid_or_semisolid_food_and TEXT, breastfeeding_children_age_623_months_receiving_an_adequate_ TEXT, nonbreastfeeding_children_age_623_months_receiving_an_adequa TEXT, total_children_age_623_months_receiving_an_adequate_diet16_1 TEXT, children_under_5_years_who_are_stunted_heightforage18_ TEXT, children_under_5_years_who_are_wasted_weightforheight18_ TEXT, children_under_5_years_who_are_severely_wasted_weightforheig TEXT, children_under_5_years_who_are_underweight_weightforage18_ TEXT, children_under_5_years_who_are_overweight_weightforheight20_ TEXT, women_age_1549_years_whose_body_mass_index_bmi_is_below_norm TEXT, men_age_1549_years_whose_body_mass_index_bmi_is_below_normal TEXT, women_age_1549_years_who_are_overweight_or_obese_bmi_250_kgm TEXT, men_age_1549_years_who_are_overweight_or_obese_bmi_250_kgm2_ TEXT, women_age_1549_years_who_have_high_risk_waisttohip_ratio_085 TEXT, men_age_1549_years_who_have_high_risk_waisttohip_ratio_090_ TEXT, children_age_659_months_who_are_anaemic_110_gdl22_ TEXT, nonpregnant_women_15_49_who_are_anaemic_120_gdl22_ TEXT, pregnant_women_15_49_who_are_anaemic_110_gdl22_ TEXT, all_women_15_49_who_are_anaemic22_ DOUBLE PRECISION, all_women_age_1519_years_who_are_anaemic22_ TEXT, men_age_1549_years_who_are_anaemic_130_gdl22_ TEXT, men_age_1519_years_who_are_anaemic_130_gdl22_ TEXT, women_age_15_years_and_above_with_high_141160_mgdl_blood_sug TEXT, women_age_15_years_and_above_wih_very_high_160_mgdl_blood_su DOUBLE PRECISION, women_age_15_years_and_above_wih_high_or_very_high_140_mgdl_ DOUBLE PRECISION, men_age_15_years_and_above_wih_high_141160_mgdl_blood_sugar_ DOUBLE PRECISION, men_age_15_years_and_above_wih_very_high_160_mgdl_blood_suga DOUBLE PRECISION, men_age_15_years_and_above_wih_high_or_very_high_140_mgdl_bl DOUBLE PRECISION, women_age_15_years_and_above_wih_mildly_elevated_blood_press DOUBLE PRECISION, women_age_15_years_and_above_wih_moderately_or_severely_elev DOUBLE PRECISION, women_age_15_years_and_above_wih_elevated_blood_pressure_sys DOUBLE PRECISION, men_age_15_years_and_above_wih_mildly_elevated_blood_pressur DOUBLE PRECISION, men_age_15_years_and_above_wih_moderately_or_severely_elevat DOUBLE PRECISION, men_age_15_years_and_above_wih_elevated_blood_pressure_systo DOUBLE PRECISION, women_age_3049_years_ever_undergone_a_screening_test_for_cer TEXT, women_age_3049_years_ever_undergone_a_breast_examination_for TEXT, women_age_3049_years_ever_undergone_an_oral_cavity_examinati TEXT, men_age_3049_yearsever_undergone_an_oral_cavity_examination_ TEXT, women_age_1549_years_who_have_comprehensive_knowledge24_of_h TEXT, men_age_1549_years_who_have_comprehensive_knowledge24_of_hiv TEXT, women_age_1549_years_who_know_that_consistent_condom_use_can TEXT, men_age_1549_years_who_know_that_consistent_condom_use_can_r TEXT, currently_married_women_age_1549_years_who_usually_participa TEXT, women_age_1549_years_who_worked_in_the_last_12_months_and_we TEXT, women_age_1549_years_owning_a_house_andor_land_alone_or_join TEXT, women_age_1549_years_having_a_bank_or_savings_account_that_t TEXT, women_age_1549_years_having_a_mobile_phone_that_they_themsel TEXT, women_age_1524_years_who_use_hygienic_methods_of_protection_ TEXT, evermarried_women_age_1849_years_who_have_ever_experienced_s TEXT, evermarried_women_age_1849_years_who_have_experienced_physic TEXT, young_women_age_1829_years_who_experienced_sexual_violence_b TEXT, women_age_15_years_and_above_who_use_any_kind_of_tobacco_ DOUBLE PRECISION, men_age_15_years_and_above_who_use_any_kind_of_tobacco_ DOUBLE PRECISION, women_age_15_years_and_above_who_consume_alcohol_ DOUBLE PRECISION, men_age_15_years_and_above_who_consume_alcohol_ DOUBLE PRECISION);
    CREATE TABLE education_stats (state BIGINT, no_of_households BIGINT, total_person BIGINT, total_male BIGINT, total_female BIGINT, population_in_the_age_group_06_person BIGINT, population_in_the_age_group_06_male BIGINT, population_in_the_age_group_06_female BIGINT, scheduled_castes_person BIGINT, scheduled_castes_male BIGINT, scheduled_castes_populationfemale BIGINT, scheduled_tribes_person BIGINT, scheduled_tribes_male BIGINT, scheduled_tribes_female BIGINT, literates_person BIGINT, literates_male BIGINT, literates_female BIGINT, illiterate_persons BIGINT, illiterate_male BIGINT, illiterate_female BIGINT, total_worker_person BIGINT, total_worker_male BIGINT, total_worker_female BIGINT, main_working_person BIGINT, main_working_male BIGINT, main_working_female BIGINT, main_cultivator_person BIGINT, main_cultivator_male BIGINT, main_cultivator_female BIGINT, main_agricultural_labourers_person BIGINT, main_agricultural_labourers_male BIGINT, main_agricultural_labourers_female BIGINT, main_household_industries_person BIGINT, main_household_industries_male BIGINT, main_household_industries_female BIGINT, main_other_workers_person BIGINT, main_other_workers_male BIGINT, main_other_workers_female BIGINT, marginal_worker_person BIGINT, marginal_worker_male BIGINT, marginal_worker_female BIGINT, marginal_cultivator_person BIGINT, marginal_cultivator_male BIGINT, marginal_cultivator_female BIGINT, marginal_agriculture_labourers_person BIGINT, marginal_agriculture_labourers_male BIGINT, marginal_agriculture_labourers_female BIGINT, marginal_household_industries_person BIGINT, marginal_household_industries_male BIGINT, marginal_household_industries_female BIGINT, marginal_other_workers_person BIGINT, marginal_other_workers_male BIGINT, marginal_other_workers_female BIGINT, marginal_worker_population_3_6_person BIGINT, marginal_worker_population_3_6_male BIGINT, marginal_worker_population_3_6_female BIGINT, marginal_cultivator_population_3_6_person BIGINT, marginal_cultivator_population_3_6_male BIGINT, marginal_cultivator_population_3_6_female BIGINT, marginal_agriculture_labourers_population_3_6_person BIGINT, marginal_agriculture_labourers_population_3_6_male BIGINT, marginal_agriculture_labourers_population_3_6_female BIGINT, marginal_household_industries_population_3_6_person BIGINT, marginal_household_industries_population_3_6_male BIGINT, marginal_household_industries_population_36_female BIGINT, marginal_other_workers_person_3_6_person BIGINT, marginal_other_workers_person_3_6_male BIGINT, marginal_other_workers_person_3_6_female BIGINT, marginal_worker_population_0_3_person BIGINT, marginal_worker_population_0_3_male BIGINT, marginal_worker_population_0_3_female BIGINT, marginal_cultivator_population_0_3_person BIGINT, marginal_cultivator_population_0_3_male BIGINT, marginal_cultivator_population_0_3_female BIGINT, marginal_agriculture_labourers_population_0_3_person BIGINT, marginal_agriculture_labourers_population_0_3_male BIGINT, marginal_agriculture_labourers_population_0_3_female BIGINT, marginal_household_industries_population_0_3_person BIGINT, marginal_household_industries_population_0_3_male BIGINT, marginal_household_industries_population_0_3_female BIGINT, marginal_other_workers_population_0_3_person BIGINT, marginal_other_workers_population_0_3_male BIGINT, marginal_other_workers_population_0_3_female BIGINT, non_working_person BIGINT, non_working_male BIGINT, non_working_female BIGINT, tru_id BIGINT);
    CREATE TABLE religion_stats (state BIGINT, tru_id BIGINT, religion_id BIGINT, tot_p BIGINT, tot_m BIGINT, tot_f BIGINT, p_06 BIGINT, m_06 BIGINT, f_06 BIGINT, p_lit BIGINT, m_lit BIGINT, f_lit BIGINT, p_ill BIGINT, m_ill BIGINT, f_ill BIGINT, tot_work_p BIGINT, tot_work_m BIGINT, tot_work_f BIGINT, mainwork_p BIGINT, mainwork_m BIGINT, mainwork_f BIGINT, main_cl_p BIGINT, main_cl_m BIGINT, main_cl_f BIGINT, main_al_p BIGINT, main_al_m BIGINT, main_al_f BIGINT, main_hh_p BIGINT, main_hh_m BIGINT, main_hh_f BIGINT, main_ot_p BIGINT, main_ot_m BIGINT, main_ot_f BIGINT, margwork_p BIGINT, margwork_m BIGINT, margwork_f BIGINT, marg_cl_p BIGINT, marg_cl_m BIGINT, marg_cl_f BIGINT, marg_al_p BIGINT, marg_al_m BIGINT, marg_al_f BIGINT, marg_hh_p BIGINT, marg_hh_m BIGINT, marg_hh_f BIGINT, marg_ot_p BIGINT, marg_ot_m BIGINT, marg_ot_f BIGINT, margwork_3_6_p BIGINT, margwork_3_6_m BIGINT, margwork_3_6_f BIGINT, marg_cl_3_6_p BIGINT, marg_cl_3_6_m BIGINT, marg_cl_3_6_f BIGINT, marg_al_3_6_p BIGINT, marg_al_3_6_m BIGINT, marg_al_3_6_f BIGINT, marg_hh_3_6_p BIGINT, marg_hh_3_6_m BIGINT, marg_hh_3_6_f BIGINT, marg_ot_3_6_p BIGINT, marg_ot_3_6_m BIGINT, marg_ot_3_6_f BIGINT, margwork_0_3_p BIGINT, margwork_0_3_m BIGINT, margwork_0_3_f BIGINT, marg_cl_0_3_p BIGINT, marg_cl_0_3_m BIGINT, marg_cl_0_3_f BIGINT, marg_al_0_3_p BIGINT, marg_al_0_3_m BIGINT, marg_al_0_3_f BIGINT, marg_hh_0_3_p BIGINT, marg_hh_0_3_m BIGINT, marg_hh_0_3_f BIGINT, marg_ot_0_3_p BIGINT, marg_ot_0_3_m BIGINT, marg_ot_0_3_f BIGINT, non_work_p BIGINT, non_work_m BIGINT, non_work_f BIGINT);
    CREATE TABLE occupation_stats (state BIGINT, tru_id BIGINT, age_group_id BIGINT, population_total BIGINT, population_male BIGINT, population_female BIGINT, main_workers_total BIGINT, main_workers_male BIGINT, main_workers_female BIGINT, marginal_workers_total BIGINT, marginal_workers_male BIGINT, marginal_workers_female BIGINT, marg_3_6mo_total BIGINT, marg_3_6mo_male BIGINT, marg_3_6mo_female BIGINT, marg_less_3mo_total BIGINT, marg_less_3mo_male BIGINT, marg_less_3mo_female BIGINT, non_workers_total BIGINT, non_workers_male BIGINT, non_workers_female BIGINT, seeking_work_total BIGINT, seeking_work_male BIGINT, seeking_work_female BIGINT);
    CREATE TABLE language_stats (state BIGINT, language_id BIGINT, tru_id BIGINT, person BIGINT, male BIGINT, female BIGINT);
    CREATE TABLE crop_stats (crop TEXT, normal_area_dafw DOUBLE PRECISION, area_sown_2025_26 DOUBLE PRECISION, area_sown_2024_25 DOUBLE PRECISION, difference_area DOUBLE PRECISION, pct_increase_decrease DOUBLE PRECISION);
    """

In [21]:

def get_started():
    torch.cuda.empty_cache()
    gc.collect()

    print("🚀 Loading Base Model & Adapter (T4-safe)...")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        device_map="auto",
        quantization_config=bnb_config,
        dtype=torch.float16,
    )

    # 🔴 MUST
    base_model.config.use_cache = False

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)


    model = PeftModel.from_pretrained(
        base_model,
        ADAPTER_PATH,
        is_trainable=False
    )
    model.eval()

    return tokenizer, model

def verify_inference(question, tokenizer, model):

    prompt = f"""### Task
Generate a SQL query to answer the following question:
{question}

### Database Schema
{schema_context}

### SQL
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    decoded_prompt = tokenizer.decode(inputs.input_ids[0])
    print(decoded_prompt)

    if "### SQL" not in decoded_prompt:
        print("❌ Prompt truncated before SQL anchor!")
        # raise ValueError("❌ Prompt truncated before SQL anchor!")

    torch.cuda.empty_cache()

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False,
            use_cache=False
        )

    result = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    )

    print("SQL Output:")
    print(result)


In [22]:
tokenizer, model = get_started()

🚀 Loading Base Model & Adapter (T4-safe)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
question = "How many Muslims live in Kerala?"
verify_inference(question,
                 tokenizer,
                 model)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ent_use_of_family_planning_methods_currently_married_wom.10 TEXT, current_use_of_family_planning_methods_currently_married_wom.11 TEXT, current_use_of_family_planning_methods_currently_married_wom.12 TEXT, current_use_of_family_planning_methods_currently_married_wom.13 TEXT, current_use_of_family_planning_methods_currently_married_wom.14 TEXT, current_use_of_family_planning_methods_currently_married_wom.15 TEXT, current_use_of_family_planning_methods_currently_married_wom.16 TEXT, current_use_of_family_planning_methods_currently_married_wom.17 TEXT, current_use_of_family_planning_methods_currently_married_wom.18 TEXT, current_use_of_family_planning_methods_currently_married_wom.19 TEXT, current_use_of_family_planning_methods_currently_married_wom.20 TEXT, current_use_of_family_planning_methods_currently_married_wom.21 TEXT, current_use_of_family_planning_methods_currently_married_wom.22 TEXT, current_use_of_family_planning_methods_currently_married_wom.23 TEXT, current_use_of_family_pl